In [1]:
import argparse
import json

import matplotlib.pyplot as plt
import numpy as np
import time
from os.path import join, exists, splitext, basename
from tqdm import tqdm
import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch
from torch.optim import lr_scheduler

from utils.utils import BaseExp
import timm

plt.style.use('ggplot')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
N = 3
M = 5

1//M == 3-1

0

In [5]:

conf = np.array([0.2, 0.5, 0.3, 0.9, 0.01])
print(np.argsort(conf))
print(np.argsort(np.argsort(conf)))

print(conf[np.argsort(np.argsort(conf))])
print(np.argsort(np.argsort(np.argsort(conf))))


[4 0 2 1 3]
[1 3 2 4 0]
[0.5  0.9  0.3  0.01 0.2 ]
[4 0 2 1 3]


In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--expid', '-i', type=str, default='000000')
parser.add_argument('--seed', type=int, default=0)
parser.add_argument('--data_path', type=str, default='data')
parser.add_argument('--static_path', type=str, default='static')
parser.add_argument('--epochs', '-e', type=int, default=10)
parser.add_argument('--batch_size', '-bs', type=int, default=8)
parser.add_argument('--model', '-m', type=str, default='efficientnet_b0')
parser.add_argument('--lr', type=float, default=1e-3)

args = parser.parse_args([])

NCLASS = 10
log_dict ={"train": np.zeros((args.epochs, 2)), "test": np.zeros((args.epochs, 2))}

transform_dict = {
        'train': transforms.Compose(
            [transforms.Resize((224, 224)),
             transforms.RandomHorizontalFlip(),
             transforms.ToTensor(),
             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225]),
             ]),
        'test': transforms.Compose(
            [transforms.Resize((224, 224)),
             transforms.ToTensor(),
             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225]),
             ])}

In [82]:
import pandas as pd
df = pd.DataFrame(np.concatenate([log_dict["train"], log_dict["test"]], axis=1))
df.columns = ["train loss", "train acc", "test loss", "test acc"]
df.to_csv("hoge.csv", index=False)

In [83]:
pd.read_csv("hoge.csv")

,train loss,train acc,test loss,test acc
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


In [69]:
json.dump({3: 3}, "hoge")

AttributeError: 'str' object has no attribute 'write'

In [57]:
dataloader = {}
data_size = {}
        
phase = "test"
test_dataset = torchvision.datasets.CIFAR10(root="data", train=False, download=True, transform=transform_dict[phase])
dataloader[phase] = torch.utils.data.DataLoader(
    test_dataset, batch_size=args.batch_size, shuffle=phase=="train", num_workers=4)        
data_size[phase] = len(test_dataset.targets)



Files already downloaded and verified
Files already downloaded and verified


In [21]:
model = timm.create_model(args.model, pretrained=True, num_classes=args.NCLASS)
model = model.to(device)


for j in range(10):


phase = "train"
train_dataset = torchvision.datasets.CIFAR10(root="data", train=True, download=True,
                                           transform=transform_dict[phase])
indices = np.random.permutation(np.arange(len(train_dataset)))
subset = Subset(train_dataset, indices[:5000*j])
dataloader[phase] = torch.utils.data.DataLoader(
    subset, batch_size=args.batch_size, shuffle=phase=="train", num_workers=4)
data_size[phase] = len(subset.indices)





optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
scheduler = lr_scheduler.MultiStepLR(optimizer,
                                     milestones=[int(0.8 * args.epochs)],
                                     gamma=0.1)
criterion = nn.CrossEntropyLoss()

start_time = time.time()
for epoch in range(args.epochs):
    print(f"epoch: {epoch+1}")
    for phase in ['train', 'test']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        epoch_loss = 0.0
        running_corrects = 0

        for xs, ys in tqdm(dataloader[phase]):
            xs = xs.to(device)
            ys = ys.to(device)

            if phase == "train":
                outputs = model(xs)
                loss = criterion(outputs, ys)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            else:
                with torch.no_grad():
                    outputs = model(xs)
                    loss = criterion(outputs, ys)

            _, preds = torch.max(outputs.data, 1)

            epoch_loss += loss.item()
            running_corrects += torch.sum(preds == ys)

        epoch_acc = running_corrects.item() / data_size[phase]
        epoch_loss /= data_size[phase]

        args.log_dict[phase][epoch, 0] = epoch_loss
        args.log_dict[phase][epoch, 1] = epoch_acc

    scheduler.step()

time_elapsed = time.time() - start_time
print(f'Training complete in {time_elapsed//60:.0f}m {time_elapsed%60:.0f}s')
print(f'Best val acc: {args.best_acc:.4f}')


def save_logs(args, epoch):
    colors = {'train': 'tab:blue', 'test': 'tab:orange'}
    plt.clf()
    for phase in ["train", "test"]:
        plt.plot(args.log_dict[phase][:epoch + 1, 0], label=phase, color=colors[phase])
    plt.legend()
    plt.xlabel('epoch')
    plt.title('loss')
    plt.savefig(f'{args.exp_path}/loss.png')
    plt.close()

    plt.clf()
    for phase in ["train", "test"]:
        plt.plot(args.log_dict[phase][:epoch + 1, 1], label=phase, color=colors[phase])
    xlabel = f'epoch (best epoch {args.best_epoch} eval acc {args.best_acc:.4f})'
    plt.legend()
    plt.xlabel(xlabel)
    plt.title('accuracy')
    plt.savefig(f'{args.exp_path}/accuracy.png')
    plt.close()

    args.cfg["acc"] = f'{args.best_acc:.3f}'
    with open(f'{args.exp_path}/cfg.json', 'w') as f:
        f.write(json.dumps(args.cfg, indent=4))




AttributeError: 'Namespace' object has no attribute 'NCLASS'